# Importing the necessary libraries and loading the malaria dataset from TFDS

Since there is only a training set available (from documentation provided with dataset), I decided to shuffle the data and split it into training, validation & testing sets from the start. 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt

[train_ds, valid_ds, test_ds] , info = tfds.load("malaria", 
                                                 split = ['train[:70%]', 'train[70%:85%]', 'train[85%:]'], 
                                                 shuffle_files=True, 
                                                 with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

ExtractError: Error while extracting C:\Users\al-kuba\tensorflow_datasets\downloads\ceb.nlm.nih.gov_proj_malaria_cell_imagesCpSVVrJBQVm1EAGSYJgFN2ZUxCZtjRh76bGSL61Dxmg.zip to C:\Users\al-kuba\tensorflow_datasets\downloads\extracted\ZIP.ceb.nlm.nih.gov_proj_malaria_cell_imagesCpSVVrJBQVm1EAGSYJgFN2ZUxCZtjRh76bGSL61Dxmg.zip (file: cell_images\Parasitized\C100P61ThinF_IMG_20150918_144104_cell_162.png) : Failed to create a NewWriteableFile: C:\Users\al-kuba\tensorflow_datasets\downloads\extracted\ZIP.ceb.nlm.nih.gov_proj_malaria_cell_imagesCpSVVrJBQVm1EAGSYJgFN2ZUxCZtjRh76bGSL61Dxmg.zip.incomplete_c8826e2124ea4996bb2b8dfffb00ee58\cell_images\Parasitized\C100P61ThinF_IMG_20150918_144104_cell_162.png : The system cannot find the path specified.
; No such process
On windows, path lengths greater than 260 characters may result in an error. See the doc to remove the limitation: https://docs.python.org/3/using/windows.html#removing-the-max-path-limitation

# Exploring the data



*   Looking at the structure of the dataset which shows that images are not a consistent size
*   Display the number of training samples provided in the overall dataset


*   Display the number of classes in the dataset
*   Dispaly the labels of the classes in the dataset


In [ ]:
print("Training Dataset structure --> " + str(train_ds))
print("Dataset Info --> " + str(info))
print ("******************************************\n")

ds_size = info.splits["train"].num_examples
ds_names = info.features["label"].names
ds_classes = info.features["label"].num_classes

print ("******************************************\n")
print ("Num of training samples: " + str(ds_size))
print ("Classes: " + str(ds_names))
print ("Num of Classes: " + str(ds_classes))

Display a few samples just to make sure that they are all different sizes which checks out

In [ ]:
for ex in train_ds.take(2):
    print (ex)

Using the provided show_examples method by tfds to see a few of the images with their labels & values

In [ ]:
# This works in Collab and my own personal Jupyter Notebook but not sure why it doesn't work in Kaggle, any ideas??
disp = tfds.show_examples(train_ds, info)

# Setting the global variables that will be used throughout the model

In [ ]:
NEW_HEIGHT = 200
NEW_WIDTH = 200
BATCH_SIZE = 32

A function to resize the images to a consistent size that can be used by the model

In [ ]:
def fix_image(ds):
  ds["image"] = tf.image.convert_image_dtype(ds["image"], tf.float32)
  ds["image"] = tf.image.resize_with_crop_or_pad(ds["image"], NEW_HEIGHT, NEW_WIDTH)
  return ds["image"], ds["label"];

Resizing the 3 datasets that will be used to train, validate and test our model

In [ ]:
fixed_train_ds = train_ds.map(fix_image).batch(BATCH_SIZE)
fixed_valid_ds = valid_ds.map(fix_image).batch(BATCH_SIZE)
fixed_test_ds = test_ds.map(fix_image).batch(BATCH_SIZE)

Displaying some of the converted images to see that their size is consistent and that we didn't corrupt the data in any way

In [ ]:
batch_image, batch_label = next(iter(fixed_train_ds))

plt.figure(figsize= (16,12))

for i in range(16):
    ax = plt.subplot(4,4,i+1)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(batch_image[i])
    if batch_label[i]:
        plt.title(ds_names[1])
    else:
        plt.title(ds_names[0])

# Building the model
- Create the input layer
- Create the convolutional/Pooling layers
  * They will half the filters at every layer the convulational layers increas
  * The minimum # of filters will be 2 if there are too many layers
- Falttening the layers so we can send them to the Dense layers
- Create the dense layers
  * A BatchNormalization Layer
  * A Dense Layer
  * A Dropout Layer
- Final Dense layer with 1 unit

In [ ]:
def build_model (dropout=0.25, input_shape=[NEW_HEIGHT, NEW_WIDTH, 3]):
    model = keras.models.Sequential()

    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    model.add(keras.layers.Conv2D(32,3, padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())            
    model.add(keras.layers.MaxPooling2D(2,2, padding="same"))

    model.add(keras.layers.Conv2D(64,3, padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())            
    model.add(keras.layers.MaxPooling2D(2,2, padding="same"))

    model.add(keras.layers.Conv2D(128,3, padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())            
    model.add(keras.layers.MaxPooling2D(2,2, padding="same"))

    model.add(keras.layers.Conv2D(256,3, padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())            
    model.add(keras.layers.MaxPooling2D(2,2, padding="same"))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(dropout))
    
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(dropout))

    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(dropout))

    model.add(keras.layers.Dense(32, activation='relu'))
 
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
                  metrics=[tf.metrics.AUC(), "accuracy"])
    return model

# Training the Model

Creating 2 callbacks to save the best model and to stop training if its stop improving after 5 epochs

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("malaria_tfds.h5", save_best_only=True)

earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model = build_model()
history = model.fit(fixed_train_ds, epochs=40, validation_data=fixed_valid_ds,  callbacks=[checkpoint_cb, earlystop_cb])

# Evaluating the model against the Test set

In [ ]:
model.load_weights("malaria_tfds.h5")
model.evaluate(fixed_test_ds)

In [ ]:
auc = history.history["auc"]
val_auc = history.history["val_auc"]

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history["auc"]))

plt.figure(figsize=(16, 12))
plt.subplot(3, 1, 1)
plt.plot(epochs_range, auc, label='Training AUC')
plt.plot(epochs_range, val_auc, label='Validation AUC')
plt.legend(loc='lower right')
plt.title('Training and Validation AUC')

plt.subplot(3, 1, 2)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(3, 1, 3)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Conclusion

* AUC is typically a better metric for logistic regression problems since the data could be skeweed towards one class due to natural occurence but the documentation provided in this dataset showed that both classes were equally represented so I decided to include accuracy also and let the numbers speak for themselves
* Regardless of which metric you use have a 95%+ AUC/Accurray would typically hint at some kind of overfitting but in this case I think the "clean" data played a major role in the outcome.  In real life you probably wouldn't get such a clean labelled data.